# 03 - Practical Session - Deep Learning

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Config

In [ ]:
config = {
    'batch_size': 128,
    'num_classes': 10,
    'epochs': 20,
}

## Dataset

In [ ]:
mnist = keras.datasets.mnist.load_data()

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist

In [ ]:
x_train_orig = np.copy(x_train)
y_train_orig = np.copy(y_train)

x_test_orig = np.copy(x_test)
y_test_orig = np.copy(y_test)

In [ ]:
rand_indx = np.random.choice(x_train.shape[0], 10, replace=False)

In [ ]:
fig, axs = plt.subplots(2,5,figsize=(10,5))
_ = [axs.flatten()[i].imshow(x_train[rand_indx[i]], cmap='gray') for i in range(10)]

## Preprocessing

Convert 28x28 dimensional 2D Array (images) into Vector with 784 components (flattening)

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

Convert from *uint8* to float32 

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

Transform value range of pixels from [0, 255] to [0, 1].

In [ ]:
x_train /= 255
x_test /= 255

In [ ]:
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

convert class vectors to binary class matrices

In [ ]:
y_train = keras.utils.to_categorical(y_train, config['num_classes'])
y_test = keras.utils.to_categorical(y_test, config['num_classes'])

## MLP network

A sequential model is a linear stack of operations.

In [ ]:
model = keras.Sequential()

Add a fully connected layer with ReLU activation function, followed by a dropout layer with probability 0.2, followed by a second  fully connected layer with ReLU activation and another dropout layer. 

In [ ]:
model.add(keras.layers.Dense(512, activation='relu', input_shape=(784,)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dropout(0.2))

Transform the output of the last layer into probabilities

In [ ]:
model.add(keras.layers.Dense(config['num_classes'], activation='softmax'))

Print a nice summary of the model

In [ ]:
model.summary()

Compile the model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Train the model

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=config['batch_size'],
                    epochs=config['epochs'],
                    verbose=1)

Evaluate the model

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Making test predictions

In [ ]:
rand_indx = np.random.choice(x_test.shape[0], 10, replace=False)
pred = model.predict_classes(x_test[rand_indx])

In [ ]:
fig, axs = plt.subplots(2,5,figsize=(20,7))
for i in range(10):
    axs.flatten()[i].imshow(x_test_orig[rand_indx[i]], cmap='gray')
    axs.flatten()[i].set_title('Prediction: {}, ground truth: {}'.format(pred[i], np.nonzero(y_test[rand_indx[i]])[0][0]))

## Visualize the features

Visualizing the features can help to identify structure. For examples, some of the weights might always be zero; hence, the corresponding features are never used for classification

In [ ]:
weights = model.get_weights()

In [ ]:
random_indx = np.random.choice(512, 10, replace=False)
vmin, vmax = weights[0].min(), weights[0].max()
fig, axs = plt.subplots(2,5,figsize=(10,6))
_ = [axs.flatten()[i].imshow(np.reshape(weights[0][:,r], (28, 28)), cmap='gray', vmin=.5*vmin, vmax=.5*vmax) 
     for i, r in enumerate(random_indx)]